<a href="https://colab.research.google.com/github/Shashank-Ojha/WordFormer/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
%load_ext autoreload
%autoreload 2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from utils.data_pipelines import read_file
from utils.data_pipelines import get_vocab
from utils.data_pipelines import train_validation_split
from utils.data_pipelines import get_batch

from utils.gpt import GPTConfig
from utils.gpt import GPT

from utils.loss import loss_fn
from utils.loss import estimate_loss

# Set a seed for reproducibility
torch.manual_seed(0)

In [3]:
# Read text file, convert each token (char) into integers, and split data into train and validation data.
text = read_file('shakespeare.txt')

# Here are all the unique characters that occur in this text
vocab, vocab_size = get_vocab(text)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
train_data, val_data = train_validation_split(data, 0.9)

In [4]:
print(data.shape)

torch.Size([1115394])


In [63]:
# ------ Hyperparameters ------

# -- Data Breakdown
batch_size = 64
block_size = 16

# -- Network Parameters
# Embedding Block
num_embeddings = vocab_size
embedding_dim = 48

# Blocks
num_blocks = 3

# Multi-Head Attention Block
num_heads = 4
kq_dim = 12
v_dim = 12

assert num_heads * v_dim == embedding_dim

# -- Training Parameters
learning_rate = 3e-4
max_iters = 5000
eval_iters = 1000

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [64]:
# Both x and y are (batch_size, block_size)
x, y = get_batch(train_data, batch_size, block_size)

In [65]:
# -- Define Network
config_dict = {
    'vocab_size': vocab_size,
    'max_seq_length': block_size,
    'embed_dim': embedding_dim,
    'num_blocks': num_blocks,
    'num_heads': num_heads,
    'kq_dim': kq_dim,
    'v_dim': v_dim,
}

config = GPTConfig(**config_dict)
print(config)

# # Input Shape (batch_size, block_size). 
# # Note that while block_size is passed to the Self Attention module, it really is
# # just an upper bound on the max block size. Any input shape of (batch_size, t) where
# # t <= block_size is valid.
model = GPT(config)

m = model.to(device)
x = x.to(device)

# Expected output (batch_size, block_size, vocab_size)
print(model(x).shape)

GPTConfig(vocab_size=65, max_seq_length=16, embed_dim=48, kq_dim=12, v_dim=12)
torch.Size([64, 16, 65])


In [66]:
# -- Training

# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

# Training Loop
for i in range(max_iters):
    # Shape = (batch_size, block_size)
    xb, yb = get_batch(train_data, batch_size, block_size)
    xb, yb = xb.to(device), yb.to(device)
    
    # Zero out the gradients
    optimizer.zero_grad()
    
    # Shape = (batch_size, block_size, vocab_size)
    logits = m(xb)
    
    # Compute loss
    loss = loss_fn(logits, yb)
    loss.backward()

    # Adjust weights
    optimizer.step()
    
    # Gather data report
    if i % eval_iters == 0 or i == max_iters - 1:
        losses = estimate_loss(m, train_data, val_data, eval_iters, batch_size, block_size, device)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        

step <built-in function iter>: train loss 4.4503, val loss 4.4403
step <built-in function iter>: train loss 2.2759, val loss 2.2887
step <built-in function iter>: train loss 2.1380, val loss 2.1747
step <built-in function iter>: train loss 2.0646, val loss 2.1116
step <built-in function iter>: train loss 2.0159, val loss 2.0788
step <built-in function iter>: train loss 1.9760, val loss 2.0511


In [67]:
# Generate Shakespeare :)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
new_text = model.generate(context, max_new_tokens=500)
print(decode(new_text[0].tolist()))


YORIOLO:
To a wropart, so then tiaBseed my puk hauspgove, I I dent spead,
Or rof s
't Ca dyou, grene; apmittre what wo
sfes for hrobury seruou, not me cre farn:
Grothe is rives vamew, and hixe ow
But me or the e nther, hey crybe mere on it
Ane a cetherr mocke ullene;
I'd weally af sell havid trivoscke wellar, Sor-gert,
et may.

LUCELIZALO:
Ford verese.

YCOLALIUS:
What sabes poord thad'd thy coulnod.
DUKENS:
Or boole; not.
Pordst Myrizeswncer mest sod the imumis, ther awe,
And tho tis deasencent
